<a href="https://colab.research.google.com/github/bonaguidin/email-parse-and-response/blob/main/Email_Responder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Core Functions

In [1]:
# Install necessary packages
!pip install openai ipywidgets streamlit pyngrok fuzzywuzzy ipython gradio  # Explanation: Installing required packages

# Clear output after installations
from IPython.display import clear_output
clear_output()  # Explanation: Clears the notebook output to keep the display clean

# Imports
import os
import openai
from IPython.display import display
import ipywidgets as widgets
import re
import streamlit as st
from pyngrok import ngrok
from fuzzywuzzy import fuzz
import gradio as gr
from ipywidgets import Textarea, Dropdown

print("✅ All libraries successfully imported!")  # Explanation: Confirmation message

# Initialize SambaNova connection
client = openai.OpenAI(
    api_key=("REPLACE_WITH_API_KEY"),  # Use SambaNova, OpenAI or API key of choice
    base_url="https://api.sambanova.ai/v1",
)

if client is not None:
    print("✅ Connection to SambaNova successful!")
else:
    print("❌ Failed to connect to SambaNova.")  # Explanation: Simple connection check message

# AI Response Templates (Used for dynamic prompt instructions)
PAST_TEMPLATES = {
    "tour_response": "Greet user, confirm tour interest, ask if they would like to schedule a tour or do so on a walk-in basis, and highlight the facilities.",
    "membership_response": "Welcome the user, recommend programs, and mention the variety of options available. For questions, direct them to call 214-328-3849.",
}

PROGRAM_DETAILS = {
    "Swim Programming": "Summer pool availability, swim lessons, pool rentals, swim team options.",
    "Group Exercise Classes": "16+ classes, free with membership, social fitness benefits.",
    "Youth Programming": "Seasonal sports (soccer, flag football, basketball, etc.).",
    "Personal Training & Nutritional Coaching": "Individual, partner, and group training options.",
    "Community": "Strong community for all ages and interests.",
    "General Fitness": "Free weight area, gym equipment, and personal trainer session.",
}

print("✅ Templates and program details loaded!")  # Explanation: Confirmation message for loaded templates

# Core functionality
def parse_structured_email(email_text):
    """Parses emails with more flexibility, extracting key information."""

    parsed = {}

    # Extract Name
    name_match = re.search(r"(?:name|first and last name)\s*:?\s*([\w\s]+)", email_text, re.IGNORECASE)
    if name_match:
        parsed['name'] = name_match.group(1).strip()
    else:
        # Fuzzy matching for name
        lines = email_text.split('\n')
        best_match_score = 0
        best_match_line = None
        for line in lines:
            score = fuzz.partial_ratio("name", line.lower())
            if score > best_match_score:
                best_match_score = score
                best_match_line = line
        if best_match_score > 75:
            name_value_match = re.search(r"([\w\s]+)", best_match_line, re.IGNORECASE)
            if name_value_match:
                parsed['name'] = name_value_match.group(1).strip()

    # Extract Tour Request
    tour_match = re.search(r"(?:tour|schedule a tour)\s*(yes|no|y|n)", email_text, re.IGNORECASE)
    if tour_match:
        parsed['tour_request'] = tour_match.group(1).lower() in ('yes', 'y')
    else:
        # Fuzzy matching for tour request
        lines = email_text.split('\n')
        best_match_score = 0
        best_match_line = None
        for line in lines:
            score = fuzz.partial_ratio("tour", line.lower())
            if score > 75:
                yes_no_match = re.search(r"(yes|no|y|n)", line.lower())
                if yes_no_match:
                    parsed['tour_request'] = yes_no_match.group(1).lower() in ('yes', 'y')

    # Extract Interests
    interests_match = re.search(r"(?:interests?)\s*:?\s*(.+)", email_text, re.IGNORECASE)
    if interests_match:
        parsed['interests'] = interests_match.group(1).strip()
    else:
        # Fuzzy matching for interests
        lines = email_text.split('\n')
        best_match_score = 0
        best_match_line = None
        for line in lines:
            score = fuzz.partial_ratio("interests", line.lower())
            if score > 75:
                interests_value_match = re.search(r"(.+)", line, re.IGNORECASE)
                if interests_value_match:
                    parsed['interests'] = interests_value_match.group(1).strip()

    print("Parsed data:", parsed)
    return parsed

def generate_response(email_text, template_choice):
    print("💡 Parsing email and generating response...")  # Explanation: Debugging message

    # Step 1: Extract User Information from Email
    try:
        parsed_data = parse_structured_email(email_text)
        full_name = parsed_data.get('name', 'Member')
        first_name = full_name.split()[0]  # Explanation: Use only the first name
        user_interests = parsed_data.get('interests', '').lower()
        wants_tour = parsed_data.get('tour_request', '').strip().lower() == 'yes'
        tour_date = parsed_data.get('tour_date', None)  # Explanation: Assuming tour_date may exist in parsed data
        phone_number = "[Phone Number]"  # Explanation: This should be updated or stored in a configuration
    except Exception as e:
        print(f"❌ Error parsing email: {str(e)}")
        return f"Error: Failed to parse email data. {str(e)}"

    # Step 2: Determine Relevant Programs Based on Interests
    relevant_programs = [PROGRAM_DETAILS[prog] for prog in PROGRAM_DETAILS if any(word in user_interests for word in prog.lower().split())]
    program_details_str = "\n".join([f"- {program}" for program in relevant_programs])

    # Step 3: Construct Dynamic AI Prompt for Email Response
    try:
        # Build a dynamic instruction prompt for the LLM
        system_prompt = f"""You are a YMCA membership coordinator. Please craft a personalized email based on the following details:
- First Name: {first_name}
- Interests: {parsed_data.get('interests', 'Not provided')}
- Tour Requested: {'Yes' if wants_tour else 'No'}
- Relevant YMCA Programs: {program_details_str if program_details_str else 'General Fitness and Well-being'}

Instructions: {PAST_TEMPLATES.get(template_choice.lower().replace(' ', '_'), 'Thank the user for submitting a web-form and provide necessary details.')}
If a tour is requested, mention the that the user can call to schedule or simply drop in on a walk-in basis. Let the user know they can view this link for additional membership information: https://ymcadallas.org/join-y
Use a warm, friendly tone.
"""
        print("✅ Dynamic prompt constructed successfully!")  # Explanation: Debug message

        # Step 4: Generate email response using AI
        response = client.chat.completions.create(
            model='Meta-Llama-3.1-8B-Instruct',  # Explanation: Model selection for generating response
            messages=[
                {"role": "system", "content": system_prompt},  # Explanation: System message with full instructions
                {"role": "user", "content": "Please generate the personalized email response."}  # Explanation: User message to trigger response generation
            ],
            temperature=0.9,  # Explanation: Controls creativity of the response
            top_p=0.7,  # Explanation: Top-p sampling parameter
            presence_penalty=0.3  # Explanation: Penalty to encourage new topics in generation
        )
        print("✅ AI response generated successfully!")
        return response.choices[0].message.content

    except Exception as e:
        print(f"❌ Error generating AI response: {str(e)}")
        return f"Error: {str(e)}"

# Gradio interface
def generate_response_gradio(email_text, template_choice):
    return generate_response(email_text, template_choice)

interface = gr.Interface(
    fn=generate_response_gradio,
    inputs=[
        gr.Textbox(lines=5, placeholder="Paste email here..."),  # Explanation: Input field for the email text
        gr.Dropdown(['Auto Select', 'Tour Response', 'Membership Response'], label="Template"),  # Explanation: Dropdown for template selection
    ],
    outputs=gr.Textbox(lines=10, label="Generated Response"),  # Explanation: Output field for the generated response
    title="YMCA Email Responder",
)

print("🚀 Launching Gradio interface...")  # Explanation: Notification before launching the interface
interface.launch(share=True)


/usr/local/lib/python3.11/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


✅ All libraries successfully imported!
✅ Connection to SambaNova successful!
✅ Templates and program details loaded!
🚀 Launching Gradio interface...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5285aa9742453ac81d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
